In [1]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

In [2]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'



In [3]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options =  dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))


In [4]:
from qiskit_metal.analyses.quantization import ScatteringImpedanceSim
em1 = ScatteringImpedanceSim(design, "hfss")

In [5]:
# example: update single setting
em1.setup.sim.max_passes = 12
em1.setup.sim.sweep_setup.stop_ghz = 13
# example: update multiple settings
em1.setup_update("sim", max_delta_s = 0.15, freq_ghz = 5.2)

em1.setup.sim

{'name': 'Setup',
 'freq_ghz': 5.2,
 'max_delta_s': 0.15,
 'max_passes': 12,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '10 nH', 'Cj': '0 fF'},
 'sweep_setup': {'name': 'Sweep',
  'start_ghz': 2.0,
  'stop_ghz': 13,
  'count': 101,
  'step_ghz': None,
  'type': 'Fast',
  'save_fields': False}}

In [6]:
q1.options.hfss_inductance='11nH'


In [7]:
em1.run(name="SingleTM", components=[], open_terminations=[], port_list=[('Q1', 'readout', 70)],
           jj_to_port=[('Q1', 'rect_jj', 50, True)], box_plus_buffer = False)

INFO 01:02PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:02PM [load_ansys_project]: 	Opened Ansys App
INFO 01:02PM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 01:02PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/connorbowman/Documents/Ansoft/
	Project:   Project5
INFO 01:02PM [connect_design]: 	Opened active design
	Design:    Qbit_hfss2 [Solution type: Eigenmode]
INFO 01:02PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 01:02PM [connect]: 	Connected to project "Project5" and design "Qbit_hfss2" 😀 

INFO 01:02PM [connect_design]: 	Opened active design
	Design:    SingleTM_hfss1 [Solution type: DrivenModal]
WARNING 01:02PM [connect_setup]: 	No design setup detected.
WARNING 01:02PM [connect_setup]: 	Creating drivenmodal default setup.
INFO 01:02PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 01:02PM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssDMSetup'>)


Finally, you can plot the various parameters. The semicolon at the end of the cell can be used to suppress the inline return.

In [8]:
em1.get_impedance();  # default: ['Z11', 'Z21']

In [9]:
em1.get_admittance();  # default: ['Y11', 'Y21']

In [10]:
em1.get_scattering();  # default: ['S11', 'S21', 'S22']

In [11]:
em1.get_data()

{'sim_setup_name': 'Setup1', 'sweep_name': 'Sweep'}

In [12]:
em1.dashboard()

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.